In [1]:
import tensorflow as tf
print(tf.__version__)

1.13.1


## Simple graph and execution

In [2]:
x = tf.Variable(3, name='x')
y = tf.Variable(4, name='y')
f = x*x*y + y + 2

sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)

print(result)

sess.close()

Instructions for updating:
Colocations handled automatically by placer.
42


In [3]:
# Create an auto-closed session

with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()

print(result)

42


In [4]:
# Global initializer

init = tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()
    result = f.eval()

print(result)

42


In [5]:
# Interactive session
# When instantiated, automatically set as the default session

sess = tf.InteractiveSession()
init.run()
result = f.eval()
print(result)
sess.close()

42


In [6]:
# Managing nodes in graphs

# Nodes are automatically added to default graph
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

In [7]:
# Can make a graph as the default temporarily and add nodes to it
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)

print(x2.graph is graph)
print(x2.graph is tf.get_default_graph())

True
False


## Linear Regression

In [8]:
import numpy as np
from sklearn.datasets import fetch_california_housing

In [9]:
# Fetch the housing data and add a bias term to the front (column of m 1s)
housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

print(housing.data.shape)
print(housing_data_plus_bias.shape)

(20640, 8)
(20640, 9)


In [10]:
X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
XT = tf.transpose(X)
theta = tf.linalg.matmul(tf.linalg.matmul(tf.matrix_inverse(tf.linalg.matmul(XT, X)), XT), y) # Normal equation

init = tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()
    theta_value = sess.run(theta)

print(theta_value)

[[-3.6959320e+01]
 [ 4.3698898e-01]
 [ 9.4245886e-03]
 [-1.0791138e-01]
 [ 6.4842808e-01]
 [-3.9986235e-06]
 [-3.7866351e-03]
 [-4.2142656e-01]
 [-4.3467718e-01]]


# Gradient Descent

In [11]:
# Set hyperparameters
n_epochs = 1000
learning_rate = 0.01

In [12]:
# Scale the housing data
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_housing_data_plus_bias = scaler.fit_transform(housing_data_plus_bias)

In [13]:
# Set up the graph ops

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name='theta')
y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
#gradients = 2/m * tf.matmul(tf.transpose(X), error) # manual differentiation
#gradients = tf.gradients(mse, [theta])[0] # automatic differentiation
#optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate) # Use TF built-in GD optimizer
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9) # TF Momentum optimizer
training_op = optimizer.minimize(mse) # Using the optimizer for the training operation
#training_op = tf.assign(theta, theta - learning_rate * gradients) # manual training operation

In [14]:
# Execute the graph

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print('Epoch', epoch, 'MSE = ', mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()

Epoch 0 MSE =  8.547485
Epoch 100 MSE =  4.8203044
Epoch 200 MSE =  4.8046446
Epoch 300 MSE =  4.803424
Epoch 400 MSE =  4.803276
Epoch 500 MSE =  4.8032565
Epoch 600 MSE =  4.8032546
Epoch 700 MSE =  4.8032537
Epoch 800 MSE =  4.8032546
Epoch 900 MSE =  4.8032546


In [15]:
print(best_theta)

[[ 0.82124686]
 [ 0.8296321 ]
 [ 0.1187541 ]
 [-0.2655507 ]
 [ 0.3057159 ]
 [-0.00450236]
 [-0.03932674]
 [-0.89985687]
 [-0.87051356]]


# Feeding Data to the Training Algorithm

In [16]:
# Fun with placeholders

A = tf.placeholder(tf.float32, shape=(None, 3)) # TF placeholder for a 2D tensor with any number of rows, 3 columns
B = A + 5

with tf.Session() as sess:
    B_val_1 = B.eval(feed_dict={A: [[1, 2, 3]]})
    B_val_2 = B.eval(feed_dict={A: [[4, 5, 6], [7, 8, 9]]})

print(B_val_1)
print(B_val_2)

[[6. 7. 8.]]
[[ 9. 10. 11.]
 [12. 13. 14.]]


In [17]:
# Mini-batch GD

X = tf.placeholder(tf.float32, shape=(None, n+1), name='X')
y = tf.placeholder(tf.float32, shape=(None, 1), name='y')

batch_size = 100
n_batches = int(np.ceil(m / batch_size))

print(m)
print(n_batches)

20640
207


In [18]:
def fetch_batch(epoch, batch_index, batch_size):
    batch_start = batch_index*batch_size
    batch_end = batch_start+batch_size
    X_batch = scaled_housing_data_plus_bias[batch_start:batch_end, :]
    y_batch = housing.target.reshape(-1,1)[batch_start:batch_end]
    
    return X_batch, y_batch

In [19]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
    
    best_theta = theta.eval()

In [20]:
best_theta

array([[-6.85214996e-04],
       [ 8.29618275e-01],
       [ 1.18751466e-01],
       [-2.65525162e-01],
       [ 3.05694878e-01],
       [-4.50308062e-03],
       [-3.93262319e-02],
       [-8.99888575e-01],
       [-8.70543718e-01]], dtype=float32)

# TensorBoard

In [25]:
# Need unique log directories for TensorBoard to read stats from

from datetime import datetime

now = datetime.utcnow().strftime('%Y%m%d%H%M%S')
root_logdir = 'tf_logs'
logdir = '{}/run-{}/'.format(root_logdir, now)

In [27]:
# Re-create the graph

# Set hyperparameters
n_epochs = 1000
learning_rate = 0.01

# Scale the housing data
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_housing_data_plus_bias = scaler.fit_transform(housing_data_plus_bias)

# Set up the graph ops
#X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
#y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
X = tf.placeholder(tf.float32, shape=(None, n+1), name='X')
y = tf.placeholder(tf.float32, shape=(None, 1), name='y')

batch_size = 100
n_batches = int(np.ceil(m / batch_size))
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name='theta')
y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9) # TF Momentum optimizer
training_op = optimizer.minimize(mse) # Using the optimizer for the training operation

# Summary variables for TensorBoard
mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
    
    best_theta = theta.eval()

file_writer.close()